<a href="https://colab.research.google.com/github/zeliasporto/rt_detr_tree_crown/blob/main/rt_detr_tree_crown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/lyuwenyu/RT-DETR.git

Cloning into 'RT-DETR'...
remote: Enumerating objects: 990, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 990 (delta 2), reused 6 (delta 1), pack-reused 978 (from 1)
Receiving objects: 100% (990/990), 622.02 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (460/460), done.


In [ ]:
import os
if not os.path.exists('individual_urban_tree_crown_detection'):
  !git clone https://github.com/pedrozamboni/individual_urban_tree_crown_detection.git
  !mkdir -p /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection

%cd /content

path = '/content/individual_urban_tree_crown_detection/rgb'

!mv {path}/* /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection
print(f'arquivos movidos de {path} para /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection')

Cloning into 'individual_urban_tree_crown_detection'...
remote: Enumerating objects: 696, done.
remote: Total 696 (delta 0), reused 0 (delta 0), pack-reused 696 (from 1)
Receiving objects: 100% (696/696), 184.86 MiB | 55.82 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (677/677), done.
/content
arquivos movidos de /content/individual_urban_tree_crown_detection/rgb para /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection


In [ ]:
%cd /content

/content


In [ ]:
import shutil

def separar_imagens(path, txt_train, txt_val, txt_test):
    os.makedirs(os.path.join(path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(path, 'val'), exist_ok=True)
    os.makedirs(os.path.join(path, 'test'), exist_ok=True)

    # Função para ler o conteúdo de um arquivo txt e retornar os nomes das imagens
    def ler_imagens(txt_file):
        with open(txt_file, 'r') as f:
            return [line.strip() for line in f.readlines()]  # Remove as quebras de linha

    # Ler os arquivos .txt
    imagens_train = ler_imagens(txt_train)
    imagens_val = ler_imagens(txt_val)
    imagens_test = ler_imagens(txt_test)

    # Função para mover as imagens para a pasta de destino apropriada
    def mover_imagens(imagens, destino):
        for imagem in imagens:
            origem_imagem = os.path.join(path, imagem)
            if os.path.exists(origem_imagem):
                shutil.move(origem_imagem, os.path.join(path, destino, imagem))
            else:
                print(f"A imagem {imagem} não foi encontrada na pasta de origem.")

    # Mover as imagens para as pastas apropriadas
    mover_imagens(imagens_train, 'train')
    mover_imagens(imagens_val, 'val')
    mover_imagens(imagens_test, 'test')

# Caminho para a pasta original onde as imagens estão
path = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection'

# Caminhos dos arquivos .txt com a lista das imagens para cada categoria
txt_train = '/content/individual_urban_tree_crown_detection/img_list/4/train.txt'
txt_val = '/content/individual_urban_tree_crown_detection/img_list/4/val.txt'
txt_test = '/content/individual_urban_tree_crown_detection/img_list/4/test.txt'

# Chama a função para separar as imagens
separar_imagens(path, txt_train, txt_val, txt_test)


A imagem 192.png não foi encontrada na pasta de origem.
A imagem 125.png não foi encontrada na pasta de origem.
A imagem 19.png não foi encontrada na pasta de origem.
A imagem 215.png não foi encontrada na pasta de origem.
A imagem 53.png não foi encontrada na pasta de origem.
A imagem 163.png não foi encontrada na pasta de origem.
A imagem 155.png não foi encontrada na pasta de origem.
A imagem 89.png não foi encontrada na pasta de origem.
A imagem 22.png não foi encontrada na pasta de origem.
A imagem 93.png não foi encontrada na pasta de origem.
A imagem 43.png não foi encontrada na pasta de origem.
A imagem 103.png não foi encontrada na pasta de origem.
A imagem 130.png não foi encontrada na pasta de origem.
A imagem 200.png não foi encontrada na pasta de origem.
A imagem 95.png não foi encontrada na pasta de origem.
A imagem 105.png não foi encontrada na pasta de origem.
A imagem 153.png não foi encontrada na pasta de origem.
A imagem 109.png não foi encontrada na pasta de origem.

In [ ]:
import json
import os
from PIL import Image

def parse_bbox_file(bbox_file, image_width, image_height):
    with open(bbox_file, 'r') as file:
        annotations = []
        for line in file:
            coordinates = list(map(float, line.split()))  # Leitura de coordenadas (x_min, y_min, x_max, y_max)
            x_min, y_min, x_max, y_max = coordinates

            # Garantir que as coordenadas estão dentro dos limites
            x_min = max(0, min(x_min, image_width - 1))
            y_min = max(0, min(y_min, image_height - 1))
            x_max = max(0, min(x_max, image_width - 1))
            y_max = max(0, min(y_max, image_height - 1))

            # Calcular largura, altura e área
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min
            area = bbox_width * bbox_height

            if bbox_width > 0 and bbox_height > 0:
                annotations.append({
                    "bbox": [x_min, y_min, bbox_width, bbox_height],
                    "area": area,
                    "iscrowd": 0
                })
        return annotations

def create_coco_annotations(img_list_dir, bbox_list_dir, output_path, category_name="Tree Crown"):
    images = []
    annotations = []
    categories = [{"id": 0, "name": category_name, "supercategory": "none"}]

    annotation_id = 1
    for image_id, image_name in enumerate(os.listdir(img_list_dir), 1):
        if image_name.endswith(".png"):  # Filtrar arquivos de imagem
            image_path = os.path.join(img_list_dir, image_name)
            bbox_file = os.path.join(bbox_list_dir, image_name.replace('.png', '.txt'))

            # Obter dimensões da imagem
            with Image.open(image_path) as img:
                width, height = img.size

            images.append({
                "id": image_id,
                "file_name": image_name,
                "width": width,
                "height": height
            })

            if os.path.exists(bbox_file):
                bboxes = parse_bbox_file(bbox_file, width, height)
                for bbox in bboxes:
                    annotations.append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": 0,
                        "bbox": bbox["bbox"],
                        "area": bbox["area"],
                        "iscrowd": bbox["iscrowd"]
                    })
                    annotation_id += 1

    coco_format = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as json_file:
        json.dump(coco_format, json_file, indent=4)
    print(f"Arquivo JSON gerado: {output_path}")

# Diretórios e caminhos
img_list_dir_train = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/train/'
img_list_dir_val = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/val/'
img_list_dir_test = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/test/'

bbox_list_dir = '/content/individual_urban_tree_crown_detection/bbox_txt'

output_path_train = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_train.json'
output_path_val = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_val.json'
output_path_test = '/content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_test.json'

# Gerar arquivos JSON
create_coco_annotations(img_list_dir_train, bbox_list_dir, output_path_train)
create_coco_annotations(img_list_dir_val, bbox_list_dir, output_path_val)
create_coco_annotations(img_list_dir_test, bbox_list_dir, output_path_test)



Arquivo JSON gerado: /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_train.json
Arquivo JSON gerado: /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_val.json
Arquivo JSON gerado: /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/annotations/instances_test.json


In [ ]:
from os.path import exists, join, basename, splitext

%cd /content
git_repo_url = 'https://github.com/lyuwenyu/RT-DETR.git'
project_name = splitext(basename(git_repo_url))[0]
!pip install torch==2.0.1
!pip install torchvision==0.15.2
!pip install onnx==1.14.0
!pip install onnxruntime==1.15.1
!pip install pycocotools
!pip install PyYAML
!pip install scipy
import sys
sys.path.append(project_name)

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show torch

Name: torch
Version: 2.0.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision, triton


In [ ]:
!python /content/RT-DETR/rtdetr_pytorch/tools/train.py -c /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r101vd_6x_coco.yml

2024-12-15 13:51:49.449338: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 13:51:49.466356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 13:51:49.487619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 13:51:49.494142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 13:51:49.509415: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!python /content/RT-DETR/rtdetr_pytorch/tools/train.py -c /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r101vd_6x_coco.yml -r /content/logs/checkpoint0195.pth --test-only


2024-12-14 15:43:22.781511: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-14 15:43:22.804236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-14 15:43:22.827129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-14 15:43:22.833824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 15:43:22.850535: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!python  /content/RT-DETR/rtdetr_pytorch/tools/infer.py -c /content/RT-DETR/rtdetr_pytorch/configs/rtdetr/rtdetr_r101vd_6x_coco.yml -r /content/logs/checkpoint0195.pth -d cuda -f /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/individual_urban_tree_crown_detection/test/104.png

2024-12-15 15:02:40.124004: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 15:02:40.144478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 15:02:40.167744: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 15:02:40.174453: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 15:02:40.190387: I tensorflow/core/platform/cpu_feature_guar